In [20]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [21]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
url = "https://www.investsite.com.br/seleciona_acoes.php"
driver.get(url)
botao_procurar = driver.find_element('xpath', '/html/body/div[1]/div[3]/div/form/input[1]')
driver.execute_script("arguments[0].click();", botao_procurar)
botao_acoestotal = driver.find_element('xpath', '/html/body/div[1]/div[3]/div[1]/fieldset/table/tbody/tr[4]/td[6]/input')
driver.execute_script("arguments[0].click();", botao_acoestotal)
select = Select(driver.find_element('xpath', '/html/body/div[1]/div[3]/div[2]/div[1]/div[1]/select'))
select = select.select_by_value('-1')
tabela = driver.find_element("xpath", '/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/table')
html_tabela = tabela.get_attribute("outerHTML")
data = pd.read_html(html_tabela)[0]
data.to_csv('screener.csv')
driver.quit()

In [22]:
screener = pd.read_csv('screener.csv')
screener.rename(columns={'Unnamed: 0': 'ticker',
                         'Unnamed: 2': 'preco',
                         'Unnamed: 8': 'pl',
                         'Unnamed: 9': 'pvpa',
                         'Unnamed: 13': 'mktcap',
                         'Unnamed: 12': 'volfin',
                         'Unnamed: 14': 'acoestotal'},inplace=True)
screener = screener.drop(['Unnamed: 0.1', 'Unnamed: 1', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
                        'Unnamed: 6', 'Unnamed: 7','Unnamed: 10', 'Unnamed: 11'], axis=1)
screener['volfin'] = screener['volfin'].str.replace('.', '').astype(int)
screener['volfin'] = pd.to_numeric(screener['volfin'], errors='coerce')
screener = screener.loc[screener['volfin'] > 50000]

screener['preco'] = pd.to_numeric(screener['preco'], errors='coerce')
screener['preco'] = screener['preco']/100

screener['pvpa'] = pd.to_numeric(screener['pvpa'], errors='coerce')
screener['pvpa'] = screener['pvpa'] / 100
screener = screener.loc[screener['pvpa'] > 0.0]

screener['pl'] = pd.to_numeric(screener['pl'], errors='coerce')
screener['pl'] = screener['pl'] / 100
screener = screener.loc[screener['pl'] > 0.0]

screener['acoestotal'] = screener['acoestotal'].str.replace('.', '').astype(int)
screener['acoestotal'] = pd.to_numeric(screener['acoestotal'], errors='coerce')

screener = screener.dropna(subset=['mktcap'])
screener['mktcap'] = screener['mktcap'].str.replace('.', '')
screener['mktcap'] = pd.to_numeric(screener['mktcap'], errors='coerce')

screener.to_csv('graham.csv')

/tmp/ipykernel_20342/772127032.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  screener['volfin'] = screener['volfin'].str.replace('.', '').astype(int)
/tmp/ipykernel_20342/772127032.py:26: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  screener['acoestotal'] = screener['acoestotal'].str.replace('.', '').astype(int)
/tmp/ipykernel_20342/772127032.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  screener['mktcap'] = screener['mktcap'].str.replace('.', '')


In [23]:
graham = pd.read_csv('graham.csv')
graham['lucro'] = (graham['mktcap'] / graham['pl']).round(2)
graham['lpa'] = (graham['lucro'] / graham['acoestotal']).round(2)
graham['vpa'] = ((graham['mktcap'] / graham['pvpa']) / graham['acoestotal']).round(2)
graham = graham.drop(['Unnamed: 0', 'volfin', 'mktcap', 'acoestotal', 'lucro'], axis=1)
graham['Preço Justo'] = np.sqrt(8.5 * graham['lpa'] * graham['vpa']).round(2)
graham['Margem (%)'] = (((graham['preco'] - graham['Preço Justo']) / graham['Preço Justo']) * 100).round(2)
graham = graham.sort_values('Margem (%)', ascending=True)
graham = graham[~graham.ticker.isin(['BRAP3', 'TASA3', 'PETR3', 'GOAU3', 'GOAU4', 'GGBR3', 'BRKM3', 'RNEW3', 'RNEW11', 'UNIP3', 'UNIP5', 'USIM3', 'DEXP4'])]
graham = graham.reset_index()
graham = graham.drop(['vpa', 'lpa', 'pl', 'pvpa', 'index'], axis=1)
graham = graham.rename(columns={'preco': 'Preço Atual', 'ticker': 'Empresa'})


graham.head(20)

,Empresa,Preço Atual,Preço Justo,Margem (%)
0,MNDL3,56.95,440.00,-87.06
1,RNEW4,1.57,9.71,-83.83
2,HBOR3,2.20,10.40,-78.85
3,HBRE3,3.82,15.41,-75.21
4,SHUL4,4.38,14.71,-70.22
5,PFRM3,3.06,8.47,-63.87
6,EUCA4,9.11,23.15,-60.65
7,LUPA3,3.86,9.52,-59.45
8,PETR4,25.85,62.69,-58.77
9,USIM5,7.66,17.68,-56.67
